# EP

### But :  Réserver plusieurs noeuds d'un cluster et lancer smpirun ep à l'aide de "execo.action.Remote" 


#### Préparation de l'env.

In [1]:
%%bash -s
git log -n 1
python --version

commit ef29601c8d8bf5de9c8b5a53e84611ab65f88674
Author: Martin Router Queen <jaiunpetitcerveau@gmail.com>
Date:   Wed May 23 11:51:02 2018 +0200

    Clean


Python 2.7.13


In [1]:
%%bash -s
pip3 install --user execo

In [2]:
import execo
from execo import *
from execo_g5k import *
from execo_engine import *
import os
import sys
import json
import time
import datetime
import re
from shutil import copy
from subprocess import check_output

Réservation noeuds

In [4]:
# Nom du job
jobname = ' '
# Nombre de noeuds
nodecount = 5
# Temps réservation
walltime = "6:0:0" 

# filters out Nancy's graphene cluster (buggy)
resources_selection = "-p \"cluster != 'graphene'\"" 

# Nancy
site = "nancy"

In [5]:
jobs = get_current_oar_jobs()
jobid = None
waiting_jobs = []
while jobs:
    j, site = jobs.pop()
    info = get_oar_job_info(j, site)
    if info['name'] == jobname:
        if info['state'] == 'Running':
            jobid = j
            print("A {} job is already running, using it. jobid is {}".format(jobname, jobid))
            break
        else:
            waiting_jobs.append(j)
if not jobid and not waiting_jobs:
    jobspec = OarSubmission(resources="/cluster=1/nodes={}".format(nodecount), walltime=walltime,
                            additional_options=resources_selection, job_type="deploy", name=jobname)
    jobid, _ = oarsub([(jobspec, site)]).pop()
    print("New job submitted, jobid is {}".format(jobid))
elif not jobid:
    print("One or more {} jobs exist ({}) but are not running.\n"
          " Connect to the frontend to see what is happening, and/or run the cell again.".format(
          jobname, ", ".join([str(j) for j in waiting_jobs])))

New job submitted, jobid is 1571100


On les range

In [9]:
nodes = get_oar_job_nodes(jobid)
nodes.sort(key=lambda n: n.address)
nodes

[Host('grimoire-1.nancy.grid5000.fr'),
 Host('grimoire-4.nancy.grid5000.fr'),
 Host('grimoire-5.nancy.grid5000.fr'),
 Host('grimoire-6.nancy.grid5000.fr'),
 Host('grimoire-7.nancy.grid5000.fr')]

In [22]:
force_redeploy = False # set to True to force redeploying the OS on the nodes in the deployment section
environment_dsc_file = 'SimGrid/examples/SMPI/EP/debian9-x64-bigdata-tutorial.yaml' # filename of the kadeploy environment file (YAML)

In [23]:
deployment = Deployment(hosts=nodes, env_file = os.path.abspath(environment_dsc_file),
                        other_options="-r ext4 --no-debug-mode")

deploy_ok, deploy_failed = deploy(deployment, check_deployed_command=not force_redeploy,
                              stdout_handlers=[sys.stdout],
                              stderr_handlers=[sys.stderr])

nancy: Deployment #D-71f0ce71-71a1-42e8-81df-37eebb574753 started
nancy: Grab the key file /home/nezzine/.ssh/authorized_keys
nancy: Grab the tarball file http://public.nancy.grid5000.fr/~pneyron/debian9-x64-bigdata-tutorial.tar.gz
nancy: Grab the postinstall file server:///grid5000/postinstalls/g5k-postinstall.tgz
nancy: Launching a deployment on grimoire-[1,4-7].nancy.grid5000.fr
nancy: Performing a Deploy[SetDeploymentEnvUntrusted] step
nancy:   switch_pxe
nancy:   reboot
nancy:    * Performing a soft reboot on grimoire-[1,4-7].nancy.grid5000.fr
nancy:   wait_reboot
nancy:   send_key_in_deploy_env
nancy:   create_partition_table
nancy:   format_deploy_part
nancy:   mount_deploy_part
nancy:   format_tmp_part
nancy:   format_swap_part
nancy: End of step Deploy[SetDeploymentEnvUntrusted] after 177s
nancy: Performing a Deploy[BroadcastEnvKascade] step
nancy:   send_environment
nancy:    * Broadcast time: 78s
nancy:   manage_admin_post_install
nancy:   manage_user_post_install
nancy:   c

nancy: Warning: Permanently added 'nancy.grid5000.fr,172.16.79.101' (ECDSA) to the list of known hosts.
nancy: Connection to nancy.grid5000.fr closed.


In [24]:
# execo.action.Remote(cmd,hosts,connection_params,process_args)
Remote_test = execo.action.Remote(cmd='ls',hosts=nodes,connection_params=None, process_args=None)
Remote_test.run().ok

True

In [31]:
Remote_install = execo.action.Remote(cmd='apt-get install g++ libboost-all-dev && wget http://gforge.inria.fr/frs/download.php/latestfile/8/SimGrid-3.18.tar.gz && tar -xvf SimGrid-3.18.tar.gz && cd SimGrid-3.18 && cmake -DCMAKE_INSTALL_PREFIX=/usr/local -Denable_smpi=on -Denable_documentation=off && make -j && make check && make install -j',hosts=nodes,connection_params={'user':'root'})

In [32]:
Remote_install.run().ok

True

In [46]:
path = "~/Stage-POLARIS/SimGrid/examples/SMPI/EP/"
execo.action.Remote(cmd='mkdir experiments_src',hosts=nodes,connection_params={'user':'root'}).run().ok

True

In [47]:
Remote_smpi_env = execo.action.Put(hosts=nodes,local_files=[path+"griffon.xml",path+"griffon_hostfile.txt",path+"ep.c",path+"nas_common.h",path+"nas_common.c",path+"Makefile"],remote_location='~/experiments_src',connection_params={'user':'root'})
Remote_smpi_env.run().ok

True

In [48]:
Remote_smpi = execo.action.Remote(cmd ='cd ~/experiments_src && make && make run',hosts=nodes,connection_params={'user':'root'})
Remote_smpi.run().ok

True

In [67]:
execo.Process(cmd='mkdir -p /home/nezzine/Stage-POLARIS/results/'+datetime.datetime.now().strftime("%Y-%m-%d")).run().ok

True

In [68]:
for i in range(0,nodecount) :
    execo.action.Get(hosts=nodes[i],remote_files=["~/experiments_src/smpi_results.txt"], local_location='~/Stage-POLARIS/results/'+datetime.datetime.now().strftime("%Y-%m-%d")+'/'+get_host_shortname(nodes[i])+'_'+datetime.datetime.now().strftime("%Y-%m-%d_%H:%M")+'.txt',connection_params={'user':'root'}).run().ok

In [ ]:
oardel(jobid)

In [ ]:
%%bash -s
git add _____.txt
git commit